In [2]:
pip install google-cloud-vision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 7.2 MB/s eta 0:00:00


In [11]:
from google.colab import files
import requests

# STEP 1: Upload the image
print("Upload the image")
uploaded = files.upload()

# STEP 2: Check if any file was uploaded
if not uploaded:
    print("⚠️ No file uploaded. Please try again.")
else:
    image_path = list(uploaded.keys())[0]

    # STEP 3: Your DeepAI API key
    DEEP_AI_API_KEY = 'your_api_key_here'  # Replace with your real key

    def moderate_image_deepai(image_path):
        with open(image_path, 'rb') as image_file:
            response = requests.post(
                "https://api.deepai.org/api/nsfw-detector",
                files={'image': image_file},
                headers={'api-key': DEEP_AI_API_KEY}
            )
        result = response.json()
        nsfw_score = result.get("output", {}).get("nsfw_score", 0)

        if nsfw_score >= 0.5:
            return {
                "ok": False,
                "nsfw_score": nsfw_score,
                "reason": "Likely inappropriate content"
            }

        return {
            "ok": True,
            "nsfw_score": nsfw_score,
            "message": "Image is safe to use"
        }

    # STEP 4: Test the image
    print(moderate_image_deepai(image_path))


Upload the image


⚠️ No file uploaded. Please try again.
